In [ ]:
import matplotlib.pyplot as plt
from css_dla import Model

In [ ]:
model = Model()


In [ ]:
# plot the grid
plt.imshow(model.grid, cmap='Set1')
plt.clim(0,10)
plt.show()

In [ ]:
model.loop(2000)

In [ ]:
plt.imshow(model.grid, cmap='Set1')
plt.title("Final grid 2")
plt.show()